### **Find paragraphs that can contain answers for the entered question**

In [ ]:
!pip install transformers
!pip install torch
!pip install spacy
!python -m spacy download en_core_web_sm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [1]:
import nltk
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

In [3]:
# upload BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertModel.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [4]:
# load text func
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text.split('\n')  # new line divides text into the paragraphs

# preprocess func
def preprocess_text(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop])

In [5]:
# vectorize text
def text_to_vector(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


In [6]:
# return 5 most reletable paragraphs
def find_relevant_paragraphs(paragraphs, question):
    question_vector = text_to_vector(preprocess_text(question)).reshape(1, -1)
    paragraph_vectors = np.array([text_to_vector(preprocess_text(p)).reshape(1, -1) for p in paragraphs]).squeeze(1)
    similarities = cosine_similarity(question_vector, paragraph_vectors).flatten()
    ranked_paragraphs = [paragraphs[i] for i in similarities.argsort()[::-1]]
    return ranked_paragraphs[:5]

In [19]:
def main():

    text = load_text('/content/text_giraffe.txt')  # here we input text with paragraphs
    question = "Where is the largest population of girrafes today"  # here we input question

    # call relevant function
    relevant_paragraphs = find_relevant_paragraphs(text, question)

    # Mapping
    for i, para in enumerate(relevant_paragraphs):
        print(f"Paragraph {i+1}: {para}\n")

# start
if __name__ == "__main__":
    main()


Paragraph 1: 	The largest populations of giraffes today live in nature reserves and reserves.

Paragraph 2: In the wild, life expectancy is about 25 years, in captivity - about 35 years.

Paragraph 3: Giraffes live in the savannas of Africa.

Paragraph 4: Therefore, giraffes have especially strong hearts.

Paragraph 5: Great height increases the load on the circulatory system, especially in relation to the blood supply to the brain.

